In [38]:
from dateutil import parser
import glob
from git import Repo
import json
import numpy as np
import os
import pandas as pd
import plotly.graph_objects as go
import requests
import shutil
# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [17]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [18]:
# download data
git_url = 'https://github.com/CSSEGISandData/COVID-19'
repo_dir = 'data'
shutil.rmtree(repo_dir)
Repo.clone_from(git_url, repo_dir)

<git.repo.base.Repo '/home/jovyan/data/.git'>

In [19]:
# read data into pandas
path = os.path.join(os.getcwd(),repo_dir,'csse_covid_19_data/csse_covid_19_daily_reports')
all_files = glob.iglob(os.path.join(path, "*.csv"))  
df = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)

In [20]:
df.sample(10)

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Latitude,Longitude,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Active,Combined_Key
30836,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,27149.0,Stevens,Minnesota,US,3/28/20 23:05,45.586130,-96.001637,0.0,"Stevens, Minnesota, US"
27114,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,27129.0,Renville,Minnesota,US,2020-03-27 22:14:55,44.728364,-94.945537,0.0,"Renville, Minnesota, US"
24708,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,19001.0,Adair,Iowa,US,2020-03-27 22:14:55,41.330756,-94.471059,0.0,"Adair, Iowa, US"
101309,NaN,NaN,NaN,2.0,1.0,0.0,NaN,NaN,42123.0,Warren,Pennsylvania,US,2020-04-21 23:30:50,41.813050,-79.269696,1.0,"Warren, Pennsylvania, US"
187105,NaN,NaN,NaN,35.0,2.0,0.0,NaN,NaN,40115.0,Ottawa,Oklahoma,US,2020-05-19 02:32:18,36.835327,-94.810119,33.0,"Ottawa, Oklahoma, US"
78660,NaN,NaN,NaN,486.0,13.0,0.0,NaN,NaN,22057.0,Lafourche,Louisiana,US,2020-04-14 23:33:31,29.524702,-90.409770,473.0,"Lafourche, Louisiana, US"
42289,NaN,NaN,NaN,15.0,0.0,0.0,NaN,NaN,56025.0,Natrona,Wyoming,US,2020-04-01 21:58:49,42.961801,-106.797885,0.0,"Natrona, Wyoming, US"
4612,NaN,Chile,2020-03-10T05:13:07,13.0,0.0,0.0,-35.6751,-71.543,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50255,NaN,NaN,NaN,17.0,0.0,0.0,NaN,NaN,45077.0,Pickens,South Carolina,US,4/4/20 23:34,34.888077,-82.727916,0.0,"Pickens, South Carolina, US"
175933,NaN,NaN,NaN,5634.0,287.0,0.0,NaN,NaN,17043.0,DuPage,Illinois,US,2020-05-16 02:32:19,41.851170,-88.086427,5347.0,"DuPage, Illinois, US"


In [21]:
# fix location and time due to schema changes since start of data collection
def fix_location(state_1,state_2,region_1,region_2,county):
    '''param: state_1: original field Province/State
       param: state_2: recent field Province_State
       param: region_1: original field Country/Region
       param: region_2: recent field Country_Region
       param: county: recent field Admin2 (US Counties)
       return: location: concatenated location string similar to Combined_Key but valid for all rows
    '''
    if not pd.isnull(state_1):
        state = state_1
    elif not pd.isnull(state_2):
        state = state_2
    else:
        state = None
    
    if not pd.isnull(region_1):
        region = region_1
    elif not pd.isnull(region_2):
        region = region_2
    else:
        region = None
    
    if not pd.isnull(county):
        county = county
    else:
        county = None
    location = ', '.join(filter(None,[county,state,region]))
    
    return location

# fix time
def fix_time(update_1, update_2):
    '''param: update_1: original update timestamp (Last Update)
       param: udpate_2: newer update timestamp (Last_Update)
       return: time: python datetime object 
    '''
    if not pd.isnull(update_1):
        time = parser.parse(update_1)
    elif not pd.isnull(update_2):
        time = parser.parse(update_2)
    else:
        time = np.nan
    
    return time

df['Location'] = df.apply(lambda x: fix_location(x['Province/State'],
                                                 x['Province_State'],
                                                 x['Country/Region'],
                                                 x['Country_Region'],
                                                 x['Admin2']), axis=1)
df['Time'] = df.apply(lambda x: fix_time(x['Last Update'], x['Last_Update']), axis=1)

In [22]:
# generate cases ('Confirmed') and deaths ('Deaths') over time grouped by location
df_by_location = df.groupby(['Location','FIPS']).agg({'Confirmed': list, 'Deaths': list, 'Time': list}).reset_index()

In [23]:
# add population columns, get data from US census API
# https://www.census.gov/data/developers/guidance/api-user-guide.html

# get population data for all US counties
response = requests.get("https://api.census.gov/data/2019/pep/population?get=POP&for=county:*&in=state:*")
pop_data = pd.DataFrame(response.json()[1:],columns=['population','state_code','county_code'])
pop_data['FIPS_str'] = pop_data.apply(lambda x: x['state_code'] + x['county_code'], axis=1)
pop_data['population'] = pop_data['population'].apply(int)

def fix_FIPS(fips):
    '''
    convert FIPS fields to string
    '''
    if not fips:
        return np.NaN
    fips_str = str(int(fips))
    if len(fips_str) < 5:
        fips_str = '0' + fips_str
        
    return fips_str

df_by_location['FIPS_str'] = df_by_location.apply(lambda x: fix_FIPS(x['FIPS']), axis=1)
df_by_location = df_by_location.merge(pop_data[['population','FIPS_str']], \
                                      how='left', \
                                      left_on='FIPS_str', \
                                      right_on='FIPS_str')


In [24]:
# fix population for NYC - data set assigns all Boroughs to Manhatten FIPS
nyc_fips = ['36005','36047','36061','36081','36085']
nyc_pop = pop_data[pop_data['FIPS_str'].isin(nyc_fips)]['population'].sum()
df_by_location.loc[df_by_location['FIPS_str']=='36061', 'population'] = nyc_pop
res = df_by_location[df_by_location['FIPS_str']=='36061']
res

,Location,FIPS,Confirmed,Deaths,Time,FIPS_str,population
2072,"New York City, New York, US",36061.0,"[9654.0, 12305.0, 14904.0, 17856.0, 21873.0, 25573.0, 29776.0, 33768.0, 37453.0, 43119.0, 47439.0, 51809.0, 57159.0, 63306.0, 67551.0, 72181.0, 76876.0, 81803.0, 87028.0, 92384.0, 98308.0, 103208.0, 106763.0, 110465.0, 118302.0, 123146.0, 127352.0, 135572.0, 138700.0, 141235.0, 144190.0, 147297.0, 145855.0, 150473.0, 155113.0, 158258.0, 160489.0, 162338.0, 164841.0, 167478.0, 169690.0, 172354.0, 174331.0, 175651.0, 176874.0, 178351.0, 180216.0, 181783.0, 183289.0, 184417.0, 185357.0, 186123.0, 187250.0, 188545.0, 190357.0, 191600.0, 192593.0, 193230.0, 193821.0, 194550.0]","[63.0, 99.0, 131.0, 199.0, 281.0, 366.0, 517.0, 678.0, 790.0, 932.0, 1139.0, 1397.0, 1584.0, 1905.0, 2256.0, 3485.0, 4009.0, 4571.0, 5150.0, 5820.0, 6367.0, 6898.0, 7349.0, 7905.0, 8455.0, 11477.0, 13202.0, 13202.0, 14451.0, 14604.0, 14887.0, 15074.0, 16388.0, 16646.0, 17126.0, 17280.0, 17515.0, 17682.0, 18076.0, 18069.0, 18399.0, 18491.0, 18925.0, 19057.0, 19067.0, 19174.0, 19626.0, 19561.0, 19818.0, 19789.0, 20056.0, 20237.0, 20316.0, 20406.0, 20476.0, 20576.0, 20720.0, 20806.0, 20887.0, 20934.0]","[2020-03-22 23:45:00, 2020-03-23 23:19:34, 2020-03-24 23:37:31, 2020-03-25 23:33:19, 2020-03-26 23:48:35, 2020-03-27 22:14:55, 2020-03-28 23:05:00, 2020-03-29 23:08:00, 2020-03-30 22:52:00, 2020-03-31 23:43:56, 2020-04-01 21:58:49, 2020-04-02 23:25:00, 2020-04-03 22:46:37, 2020-04-04 23:34:00, 2020-04-05 23:06:45, 2020-04-06 23:22:00, 2020-04-07 23:04:49, 2020-04-08 22:51:58, 2020-04-09 23:02:37, 2020-04-10 22:54:07, 2020-04-11 22:45:33, 2020-04-12 23:18:00, 2020-04-13 23:07:54, 2020-04-14 23:33:31, 2020-04-15 22:56:51, 2020-04-16 23:30:51, 2020-04-17 23:30:52, 2020-04-18 22:32:47, 2020-04-19 23:41:01, 2020-04-20 23:36:47, 2020-04-21 23:30:50, 2020-04-22 23:30:53, 2020-04-24 03:30:50, 2020-04-25 06:30:53, 2020-04-26 02:30:51, 2020-04-27 02:30:52, 2020-04-28 02:30:51, 2020-04-29 02:32:29, 2020-04-30 02:32:27, 2020-05-01 02:32:28, 2020-05-02 02:32:27, 2020-05-03 02:32:28, 2020-05-04 02:32:28, 2020-05-05 02:32:34, 2020-05-06 02:32:31, 2020-05-07 02:32:28, 2020-05-08 02:32:32, 2020-05-09 02:32:35, 2020-05-10 02:32:30, 2020-05-11 02:32:30, 2020-05-12 03:32:27, 2020-05-13 03:32:26, 2020-05-14 03:32:28, 2020-05-15 02:33:02, 2020-05-16 02:32:19, 2020-05-17 02:32:32, 2020-05-18 02:32:21, 2020-05-19 02:32:18, 2020-05-20 02:32:19, 2020-05-21 02:32:48]",36061,8336817.0


In [25]:
# tuncate to minimum case count
MIN_CASE_COUNT = 100
def threshold(s, min_case_count):
    
    for idx, cnt in enumerate(s['Confirmed']):
        if cnt > min_case_count:
            idx = idx-1
            break
    s['Confirmed'] = s['Confirmed'][idx+1:]
    s['Deaths'] = s['Deaths'][idx+1:]
    s['Time'] = s['Time'][idx+1:]
    return s

df_by_location = df_by_location.apply(lambda x: threshold(x,MIN_CASE_COUNT), axis=1)

In [42]:
# plotting
def add_traces(row, col, fig, do_diff, pop_upper_thresh, pop_lower_thresh):
    y = row[col]
    pop = row['population']
    y = [1000000*cases/pop for cases in y ]
    time = row['Time']
    location = row['Location']
    
    if do_diff:
        y = np.diff(y)
    
    if (pop > pop_lower_thresh) and (pop < pop_upper_thresh):
        fig.add_trace(go.Scatter(y=y,
                        mode='lines+markers',
                        name=location,
                        text=time))
    return fig

In [46]:
plot_col = 'Confirmed'
#plot_col = 'Deaths'

fig = go.Figure()
for index, row in df_by_location.iterrows():
    fig = add_traces(row, plot_col, fig, False, np.inf, 500000)

fig.update_layout(
    title={
        'text': "COVID-19 Large US Counties",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis_title="Days Since 100 Cases",
    yaxis_title="Cases per Million",
    font=dict(
        family="Courier New, monospace",
        size=10,
        color="#7f7f7f"
    )
)

In [47]:
plot_col = 'Confirmed'

fig = go.Figure()
for index, row in df_by_location.iterrows():
    fig = add_traces(row, plot_col, fig, False, 500000, 100000)
    
fig.update_layout(
    title={
        'text': "COVID-19 Medium US Counties",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis_title="Days Since 100 Cases",
    yaxis_title="Cases per Million",
    font=dict(
        family="Courier New, monospace",
        size=10,
        color="#7f7f7f"
    )
)